In [1]:
using Dates

In [2]:
function read_input(input_data)
    """
    Read in the input data to a value for N, E
    and an array of edges.
    """

    input_strings = split(input_data, "\n")
    N, E = split(input_strings[1], " ")
    N = parse(Int64, N)
    E = parse(Int64, E)
    edges = Array{Int64}(undef, 0, 2)
    new_edges = []
    # loop through and generate a complete list of edges
    for i in 2:E+1
        edge_start, edge_end = split(input_strings[i], " ")
        edge_start = parse(Int64, edge_start)
        edge_end = parse(Int64, edge_end)
        edges = vcat(edges, transpose([edge_start, edge_end]))
    end
    
    return N, E, edges
end

read_input (generic function with 1 method)

In [8]:
mutable struct Node
    """
    Node to store which node has which 
    neighbours.
    """
    node_number::Int64
    edges::Array
    colour::Int64
end


function generate_nodes(N::Int64, E::Int64, edges::Array)
    """
    Generate a complete list of nodes.
    """
    nodes = Dict{Int64, Any}()
    for i in 0:(N-1)
        nodes[i] = Node(i, [], -1)
    end

    # set up the connections between the nodes
    for i in 1:E
        edge_start, edge_end = edges[i, :]
        push!(nodes[edge_start].edges, edge_end)
        push!(nodes[edge_end].edges, edge_start)
    end
    return nodes
end


function find_domain(node, nodes, N_colours = 2)
    """
    Find the domain of a node. This should be 
    all of the colours our node can take plus
    one extra colour which we may choose.
    """
    domain = collect(1:N_colours+1)
    
    for neighbour_number in node.edges
        filter!(x -> x != nodes[neighbour_number].colour, domain)
    end     
    
    return domain
end


function find_N_colours(nodes)
    """
    Find whichever is the max colour that was
    used in the graph colouring.
    """
    max_colour = 0
    
    for (key, node) in nodes
        if node.colour > max_colour
            max_colour = node.colour
        end
    end
    return max_colour
end
        

function select_next_node(nodes; N_colours = 1)
    """
    Given a set of nodes, some of which are uncoloured,
    select the next most constrained node.
    """
    
    smallest_domain = Inf
    largest_n_neighbours = -Inf
    most_constrained_node = nothing
    
    for (key, node) in nodes
        domain = find_domain(node, nodes, N_colours)
        neighbours = node.edges
        # find node with smallest domain and most neighbours
        domain_size = length(domain)
        n_neighbours = length(neighbours)
        
        if node.colour == -1 # uncoloured node
            if domain_size <= smallest_domain # has smallest domain
                if n_neighbours > largest_n_neighbours # has the most neighbours
                    most_constrained_node = node
                    smallest_domain = domain_size
                    largest_n_neighbours = n_neighbours
                end
            end
        end
    end
    return most_constrained_node
end


function fill_greedily(nodes, N; verbose = true)
    # fill up the nodes greedily
    N_colours = 1
    choice_cache = []
    
    for i in 1:N
        node = select_next_node(nodes, N_colours = N_colours)
        # assign the smallest possible colour
        make_choice(node, nodes, N_colours, choice_cache, verbose = false)
        # re-calculate the domain size
        N_colours = find_N_colours(nodes)
    end
    
    if verbose == true
        N_colours = find_N_colours(nodes)
        println("Graph filled. Result has $(N_colours) colours.")
    end
    
    return nodes, choice_cache
end

fill_greedily (generic function with 2 methods)

In [19]:
# add in abilities to revert to any previously made choices and try again?
choice_cache = []

mutable struct Choice
    nodes::Dict # the node status at the moment the choice is made
    node # the node number we made the choice on
    choice_counter::Int64 # which choice we made
end


function make_choice(node, nodes, N_colours, choice_cache::Array; verbose = true)
    """
    Whenever a choice is made, create an instance where 
    we store the status of the system at the time of the 
    choice.
    """
    # first, open up a choice instance
    domain = find_domain(node, nodes, N_colours)
    colour = minimum(domain)
    node.colour = colour
    
    if length(domain) > 1
        # make a copy at the point of the choice
        node_copy = deepcopy(nodes)
        choice = Choice(node_copy, node_copy[node.node_number], 1)
        push!(choice_cache, choice)
    end
    
    if verbose == true
        println("Made choice of colour $(colour) for node $(node.node_number).")
    end
end

    
function revert_to_previous_choice(choice_cache)
    """
    Revert our algorithm to a previous choice and try again;
    if we run out of choices, then we are optimal. 
    """
    choice_made = false
    node = nothing
    nodes = nothing
        
    while (length(choice_cache) >= 1) & (choice_made == false)
        choice = choice_cache[end]
        choice_cache = choice_cache[1:end-1]
        nodes = choice.nodes
        node = choice.node
        choice.choice_counter += 1
        counter = choice.choice_counter
        
        # find the domain
        N_colours = find_N_colours(nodes)
        domain = find_domain(node, nodes, N_colours)
        
        # if our counter exceeds our choices, then remove.
        if length(domain) >= counter
            node.colour = domain[counter]
            choice_made = true
        end
    end
    
    return nodes, choice_cache
end


function count_uncoloured(nodes)
    N_uncoloured = 0
    for (key,node) in nodes
        if node.colour == -1
            N_uncoloured += 1
        end
    end
    return N_uncoloured
end


function refill_greedily(nodes, choice_cache, best_N; verbose = true)
    """
    Refill in a greedy fashion using a previous
    choice we made.
    """
    N = count_uncoloured(nodes)
    i = 1
    N_colours = find_N_colours(nodes)
    
    while (i < N) & (N_colours < best_N)
        node = select_next_node(nodes, N_colours = N_colours)
        # assign the smallest possible colour
        make_choice(node, nodes, N_colours, choice_cache, verbose = false)
        # re-calculate the domain size
        N_colours = find_N_colours(nodes)
        i += 1
    end
    
    if verbose == true
        N_colours = find_N_colours(nodes)
        N_uncoloured = count_uncoloured(nodes)
        println("Refilled colours. New solution has $(N_colours) in total.")
        println("New solution has $(N_uncoloured) left uncoloured.")
    end
    
    N_uncoloured = count_uncoloured(nodes)
    if N_uncoloured > 0
        feasible = false
    else
        feasible = true
    end
    
    return nodes, choice_cache, feasible
end 

refill_greedily (generic function with 1 method)

In [20]:
function format_result(result, N, optimal, N_colours)
    
    result_string = "$(N_colours) $(optimal) \n"
    for i in 0:N-1
        result_string *= " $(result[i].colour)"
    end
    return result_string
end

format_result (generic function with 1 method)

In [21]:
function colour_graph(input_data; verbose = false, iter_limit = 100000, timeout = 10)

    # start the clock...
    t0 = Dates.now()
    t1 = Dates.now()
    max_time = Dates.Millisecond(1000 * timeout)
    
    N, E, edges = read_input(input_data)
    nodes = generate_nodes(N, E, edges)
    result, choice_cache = fill_greedily(nodes, N, verbose = verbose)
    N_colours = find_N_colours(result)

    i = 1
    while (i < iter_limit) & (length(choice_cache) > 0) & (t1 - t0 < max_time)
        nodes, choice_cache = revert_to_previous_choice(choice_cache)
        nodes, choice_cache, feasible = refill_greedily(nodes, choice_cache, N_colours, verbose = verbose)
        N_colours_new = find_N_colours(nodes)
        if (N_colours_new < N_colours) & (feasible == true)
            N_colours = N_colours_new
            result = nodes
        end
        i += 1
        t1 = Dates.now()
    end

    if length(choice_cache) == 0
        optimal = 1
    else
        optimal = 0
    end
    
    formatted_result = format_result(result, N, optimal, N_colours)
    return formatted_result
end

colour_graph (generic function with 1 method)

In [22]:
input_data = read("data/gc_20_9", String)
result = colour_graph(input_data)

"11 1 \n 3 9 2 8 3 7 4 5 7 4 10 1 11 6 5 11 5 9 2 6"